# README.md

# Research question: for these 399 pts what are the drug use Y/N status for each category

## Table of Contents
* [Read data, clean up and get stats](#first-bullet)
* [Assign drug use status per row](#second-bullet)

# Read data, clean up and get stats <a class="anchor" id="first-bullet"></a>

In [1]:
import numpy as np
import pandas as pd
import sys, getopt
import csv
from datetime import datetime
import itertools

import matplotlib.pyplot as plt
from matplotlib import pyplot
from pylab import *

%matplotlib inline  
import seaborn as sns

In [14]:
df1=pd.read_csv("full_patient_level_202103021234_fixed_lost_pt_list.csv",header=0)
df2=pd.read_excel("pt_list_all_rows.xlsx", header=0) 

In [15]:
result_row_list=df2["person_id"].to_list()
df2[:5]
#print(len(result_pt_list))
print(len(result_row_list))
unique_result_pt_list=set(result_row_list)
print(len(unique_result_pt_list))

899
399


In [16]:
df1[:3]

,person_id,cancer_drug,category,start_date_source,start_date_note_id,start_date_note_date,start_date_sent_id,start_date_evidence,start_date_matched_text,start_date_year,...,ajcc_pathological_t,ajcc_pathological_n,ajcc_pathological_m,overall_stage,histologic_type_id,status,created_at,updated_at,primary_site_id,edited_attributes
0,2,Abiraterone,Hormone Therapy,Notes,10942894.0,2013-09-04,285738001.0,Noted to have CRPC 8/28/13 thus started on Abi...,abiraterone,2012.0,...,Not Reported,Not Reported,Not Reported,IV,101,reviewed,2019-08-20 21:56:15,2019-08-21 13:09:23,24,NaN
1,2,Docetaxel,Chemotherapy,MedTable,NaN,NaN,NaN,DOCETAXEL INFUSION,docetaxel,2016.0,...,Not Reported,Not Reported,Not Reported,IV,101,reviewed,2019-08-20 21:56:15,2019-08-21 13:09:23,24,NaN
2,2,Leuprolide,Hormone Therapy,Notes,10942894.0,2013-09-04,286863310.0,"By 2/10/09, his PSA had risen to 7.03 thus res...",lupron,2009.0,...,Not Reported,Not Reported,Not Reported,IV,101,reviewed,2019-08-20 21:56:15,2019-08-21 13:09:23,24,NaN


In [17]:
index_list=df1.columns
index_list

Index(['person_id', 'cancer_drug', 'category', 'start_date_source',
       'start_date_note_id', 'start_date_note_date', 'start_date_sent_id',
       'start_date_evidence', 'start_date_matched_text', 'start_date_year',
       'start_date_month', 'start_date_day', 'end_date_source',
       'end_date_note_id', 'end_date_note_date', 'end_date_sent_id',
       'end_date_evidence', 'end_date_matched_text', 'end_date_year',
       'end_date_month', 'end_date_day', 'first_mentioned_note_id',
       'first_mentioned_note_date', 'first_mentioned_evidence',
       'first_mentioned_matched_text', 'last_mentioned_note_id',
       'last_mentioned_note_date', 'last_mentioned_evidence',
       'last_mentioned_text', 'cancer_diagnosis_id', 'person_id.1',
       'cancer_type_id', 'year_of_diagnosis', 'month_of_diagnosis',
       'day_of_diagnosis', 'ajcc_clinical_t', 'ajcc_clinical_n',
       'ajcc_clinical_m', 'ajcc_pathological_t', 'ajcc_pathological_n',
       'ajcc_pathological_m', 'overall_stage',

In [18]:
#delete empty columns
print(df1.shape)
del_col=['start_date_source','start_date_note_id', 'start_date_note_date', 'start_date_sent_id',
       'start_date_evidence', 'start_date_matched_text', 'start_date_year',
       'start_date_month', 'start_date_day', 'end_date_source',
       'end_date_note_id', 'end_date_note_date', 'end_date_sent_id',
       'end_date_evidence', 'end_date_matched_text', 'end_date_year',
       'end_date_month', 'end_date_day', 'first_mentioned_note_id',
       'first_mentioned_note_date', 'first_mentioned_evidence',
       'first_mentioned_matched_text', 'last_mentioned_note_id',
       'last_mentioned_note_date', 'last_mentioned_evidence',
       'last_mentioned_text', 'cancer_diagnosis_id', 'person_id.1',
       'cancer_type_id', 'year_of_diagnosis', 'month_of_diagnosis',
       'day_of_diagnosis', 'ajcc_clinical_t', 'ajcc_clinical_n',
       'ajcc_clinical_m', 'ajcc_pathological_t', 'ajcc_pathological_n',
       'ajcc_pathological_m', 'overall_stage', 'histologic_type_id', 'status',
       'created_at', 'updated_at', 'primary_site_id', 'edited_attributes']

for col in del_col:
    del df1[col]
print(df1.shape)

(6993, 48)
(6993, 3)


In [19]:
#for this list of rows get the dx etc info 
result_row_list[:5]

[339, 345, 345, 345, 435]

In [35]:
adt=['Goserelin','Histrelin','Leuprolide','Triptorelin','Degarelix']
sec_gen=['Abiraterone', 'Enzalutamide','Apalutamide','Daralutamide']
chemo=['Docetaxel','Cabazitaxel']
sipu=['Sipuleucel-T']
raduium=['Radium-223 Dichloride']
parpi=['Olaparib','Rucaparib','Talazoparib']
systemic=adt+sec_gen+chemo+sipu+raduium+parpi

In [36]:
systemic

['Goserelin',
 'Histrelin',
 'Leuprolide',
 'Triptorelin',
 'Degarelix',
 'Abiraterone',
 'Enzalutamide',
 'Apalutamide',
 'Daralutamide',
 'Docetaxel',
 'Cabazitaxel',
 'Sipuleucel-T',
 'Radium-223 Dichloride',
 'Olaparib',
 'Rucaparib',
 'Talazoparib']

# Assign drug code status <a class="anchor" id="second-bullet"></a>

In [38]:
y=11
df1_pt=df1[(df1["person_id"] == y)]
pt_drug_list=df1_pt.cancer_drug.to_list()
print(pt_drug_list)

print(sys_stat)

[]
False


In [52]:
result_status_list=[]

for y in result_row_list: 
    
    df1_pt=df1[(df1["person_id"] == y)]
    pt_drug_list=df1_pt.cancer_drug.to_list()
    
    if len(pt_drug_list)==0:
        sys_stat='NaN'
        adt_stat='NaN'
        sec_gen_stat='NaN'
        chemo_stat='NaN'
        sipu_stat='NaN'
        raduium_stat='NaN'
        parpi_stat='NaN'
    
    if len(pt_drug_list)>0:
        sys_stat=bool(set(pt_drug_list) & set(systemic))  
        adt_stat=bool(set(pt_drug_list) & set(adt))      
        sec_gen_stat=bool(set(pt_drug_list) & set(sec_gen))    
        chemo_stat=bool(set(pt_drug_list) & set(chemo))
        sipu_stat=bool(set(pt_drug_list) & set(sipu))
        raduium_stat=bool(set(pt_drug_list) & set(raduium))
        parpi_stat=bool(set(pt_drug_list) & set(parpi))
    
    result_pt=[y,sys_stat,adt_stat,sec_gen_stat,chemo_stat,sipu_stat,raduium_stat,parpi_stat]
    result_status_list.append(result_pt)    

In [53]:
df_results=pd.DataFrame(result_status_list)
df_results.columns=(['person_id', 'Received systemic therapy (Y/N)', 'Received ADT (Y/N)', 'Received 2nd generation antiandrogens (Y/N)', 'Received Chemotherapy (Y/N)', 'Received Sipuleucel-T (Y/N)', "eceived Radium 223 (Y/N)", "Received PARP inhibitors (Y/N)"])

In [54]:
df_results = df_results.applymap(str)
df_results = df_results.replace({'True': 'Y'}, regex=True)
df_results = df_results.replace({'False': 'N'}, regex=True)

df_results[:5]

,person_id,Received systemic therapy (Y/N),Received ADT (Y/N),Received 2nd generation antiandrogens (Y/N),Received Chemotherapy (Y/N),Received Sipuleucel-T (Y/N),eceived Radium 223 (Y/N),Received PARP inhibitors (Y/N)
0,339,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,345,Y,Y,Y,Y,N,N,N
2,345,Y,Y,Y,Y,N,N,N
3,345,Y,Y,Y,Y,N,N,N
4,435,Y,Y,N,N,N,N,N


In [55]:
df_results.to_csv("pca_pts_400pts_Systemic_status_All_Rows_Mar2_2021_newtale_fixed_lost_ptlist.csv", encoding='utf-8', index=False)

Sipuleucel-T
Leuprolide
Radium-223 Dichloride

Bicalutamide
Docetaxel
Goserelin
Histrelin
Triptorelin
Abiraterone

Enzalutamide
Apalutamide

Olaparib
Rucaparib
Talazoparib